In [1]:
import pandas as pd
import mysql.connector as sql
import math

In [2]:
def get_all_bonds_in_list(start_date,end_date):
    print('start to get data from {} to {}'.format(start_date,end_date))
    #####Create engine:
    db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
    ##security query
    security_query = "select * from security_info"
    ##read data
    security_data = pd.read_sql(security_query,con=db_connection)
    security_data = security_data[['SecurityID','Currency','IssueDate','MaturityDate']]
    ##price query
    price_query = "select * from bond_spread where  KeyDate between '{}' and '{}'".format(start_date,end_date)
    #####read bond_data from db:
    final_data = pd.read_sql(price_query,con=db_connection)
    final_data = final_data.merge(security_data, on=['SecurityID'], how='left')
    final_data.dropna(subset=["ZSpread"],inplace=True)
    final_data_1 = final_data[final_data.Currency == 'USD']
    final_data = final_data_1.groupby('SecurityID')
    bonds_list = [final_data.get_group(x) for x in final_data.groups]
    db_connection.close()
    return bonds_list

In [3]:
def merge_left_right(data1,data2):
    temp_data = data1.append(data2)
    return temp_data

def merg_sort(list_data):
    list_length = len(list_data)
    if list_length == 1:
        output = list_data[0][2]
        return output
    else:
        mid_point = math.floor(list_length/2)
        #print(len(list_data[:mid_point]))
        left = list_data[mid_point:]
        right = list_data[:mid_point]
        return merge_left_right(merg_sort(left), merg_sort(right))

In [4]:
silding_windows = [['2018-12-31','2019-02-28'],['2019-03-01','2019-04-30'],
                   ['2019-05-01','2019-06-30'],['2019-07-01','2019-08-31'],
                   ['2019-09-01','2019-10-31'],['2019-11-01','2019-12-31']]

## Cluster data

In [5]:
cluster_data = pd.read_csv('Cluster_group.csv')

In [6]:
cluster_data.drop(columns=['Unnamed: 0'],inplace=True)

In [7]:
cluster_data.head()

,SecurityID,2019-02-28,2019-04-30,2019-06-30,2019-08-31,2019-10-31,2019-12-31
0,109.0,Cluster 0,Cluster 10,Cluster 1,Cluster 7,Cluster 21,Cluster 12
1,135.0,Cluster 0,Cluster 6,Cluster 3,Cluster 21,Cluster 5,Cluster 11
2,160.0,Cluster 0,Cluster 23,Cluster 1,Cluster 16,Cluster 22,Cluster 12
3,267.0,Cluster 0,Cluster 9,Cluster 4,Cluster 16,Cluster 15,Cluster 12
4,339.0,Cluster 0,Cluster 5,Cluster 6,Cluster 19,Cluster 5,Cluster 1


In [8]:
def get_the_daily_spread_windows(bonds_list):
    bond_spread_list = []
    for bond in bonds_list:
        try:
            bond['G_change']=bond.GSpread.pct_change()
            bond.dropna(subset=['G_change'],inplace = True)
            bond_spread_list.append((len(bond.GSpread.values),bond.SecurityID.iloc[0],bond))
        except:
            pass
    return bond_spread_list

In [9]:
def Average_daily_Gspread_change_cluster(window,cluster_data):
    bond_list1 = get_all_bonds_in_list(window[0],window[1])
    daily_spread_1 = get_the_daily_spread_windows(bond_list1)
    new_data = merg_sort(daily_spread_1)
    new_data.dropna(inplace=True)
    data1 = new_data.merge(cluster_data[['SecurityID',window[1]]],on=['SecurityID'],how ='left')
    temp = data1.groupby(by=['KeyDate',window[1]],as_index=False).mean()
    data1['Group'] = data1[window[1]] 
    data1.drop(columns=[window[1]],inplace=True)
    temp['Group'] = temp[window[1]]
    end = temp[['Group','KeyDate','G_change']]
    return end,data1

In [10]:
from sqlalchemy import create_engine

In [12]:
for i in range(0,len(silding_windows)):
    if i == 0:
        final_change,final = Average_daily_Gspread_change_cluster(silding_windows[i],cluster_data)
    else:
        temp,temp_data = Average_daily_Gspread_change_cluster(silding_windows[i],cluster_data)
        final_change.append(temp,ignore_index=True)
        final.append(temp_data,ignore_index=True)

start to get data from 2018-12-31 to 2019-02-28
start to get data from 2019-03-01 to 2019-04-30
start to get data from 2019-05-01 to 2019-06-30
start to get data from 2019-07-01 to 2019-08-31
start to get data from 2019-09-01 to 2019-10-31
start to get data from 2019-11-01 to 2019-12-31


In [13]:
final_change.head()

,Group,KeyDate,G_change
0,Cluster 0,2019-01-02,0.017571
1,Cluster 1,2019-01-02,0.017209
2,Cluster 10,2019-01-02,0.014612
3,Cluster 11,2019-01-02,0.026336
4,Cluster 12,2019-01-02,0.012627


In [14]:
final.head()

,SecurityID,KeyDate,Price,YieldWorst,ModifiedDuration_Plain,WorkoutDate,AmtOutstanding,GSpread,OAS_Spread,ZSpread,AswUsd,Currency,IssueDate,MaturityDate,G_change,Group
0,62650,2019-01-28,100.362,6.56471,0.941645,2020-01-27,200000000.0,397.02,367.779,379.397,372.792,USD,2019-01-29,2020-01-27 00:00:00,-0.000478,NaN
1,62650,2019-01-29,100.246,6.80844,0.941076,2020-01-27,200000000.0,422.16,387.983,405.557,399.980,USD,2019-01-29,2020-01-27 00:00:00,0.063322,NaN
2,62650,2019-01-30,100.263,6.77007,0.938578,2020-01-27,200000000.0,427.97,385.729,403.238,397.452,USD,2019-01-29,2020-01-27 00:00:00,0.013763,NaN
3,62650,2019-01-31,100.316,6.64465,0.931103,2020-01-27,200000000.0,410.11,379.871,391.700,386.778,USD,2019-01-29,2020-01-27 00:00:00,-0.041732,NaN
4,62650,2019-02-28,100.387,6.30099,0.854065,2020-01-27,200000000.0,376.88,353.846,360.896,348.068,USD,2019-01-29,2020-01-27 00:00:00,-0.081027,NaN


In [15]:
len(final)

528333

In [16]:
final = final[['SecurityID','KeyDate','YieldWorst','ModifiedDuration_Plain','AmtOutstanding','G_change','Group']]

In [17]:
final = final.rename(columns={"G_change": "Target_G_change"})

In [140]:
final.head()

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,G_change
2199,56535,2019-01-02,26.9747,1.85247,300000000.0,0.002132,0.014612
2200,56535,2019-01-03,26.7432,1.84797,300000000.0,-0.005914,0.007046
2201,56535,2019-01-04,26.9556,1.84352,300000000.0,0.006400,0.017257
2202,56535,2019-01-07,27.0412,1.84036,300000000.0,-0.001467,-0.045289
2203,56535,2019-01-08,26.9041,1.83947,300000000.0,-0.007478,-0.005836


In [19]:
final = final.merge(final_change,on=['Group','KeyDate'],how = 'left')

In [20]:
final = final.dropna()

## Read Your secuirty data here

In [52]:
db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
security_query = "select * from security_info"
security_data = pd.read_sql(security_query,con=db_connection)

## END

In [53]:
security_data.head()

,SecurityID,OfficialName,SecurityType,SecuritySubType,SecurityBBGType,SecurityBBGYellowKey,Series,SecuritySector,SecurityIndustry,SecuritySubIndustry,...,IsPerpetual,IsJunior,Seniority,PaymentRank,IsBailIn,QuoteSize,AmountIssued,MinIncrement,MinPiece,BBGDescription
0,62,BACR 4.875 08/13/19,Bond,Bond,EURO MTN,Corp,None,Financials,Banks,Commer Banks Non-US,...,0,0,Sr Unsecured,Sr Unsecured,N,100,2.000000e+09,1000.0,50000.0,None
1,64,POST 5 08/15/26,Bond,Bond,PRIV PLACEMENT,Corp,None,Consumers,Food & Beverage,Food-Misc/Diversified,...,0,0,Sr Unsecured,Sr Unsecured,Not Applicable,100,1.750000e+09,1000.0,2000.0,None
2,69,ARNDTN 1.5 05/03/22,Bond,Bond,EURO-ZONE,Corp,None,Real Estate,Real Estate,Real Estate Oper/Develop,...,0,0,Sr Unsecured,Sr Unsecured,Not Applicable,100,6.000000e+08,100000.0,100000.0,COUPON STEP UP PROVISION IF ISSUER IS NOT BBB ...
3,72,RBS 6.1 06/10/2023 Bullet USD SUBORDINATE,Bond,Bond,GLOBAL,Corp,None,Financials,Diversified Banks,Diversified Banking Inst,...,0,0,Subordinated Unsecured,Subordinated,Y,100,1.000000e+09,1000.0,2000.0,Basel III grandfathered instrument
4,78,AER 5 10/01/21,Bond,Bond,GLOBAL,Corp,None,Financials,Commercial Finance,Finance-Leasing Compan,...,0,0,Sr Unsecured,Sr Unsecured,N,100,8.000000e+08,1000.0,150000.0,ISS'D IN EXCH OF 144A/REGS SEC; SEE CUSIP# 007...


In [54]:
len(security_data)

28976

In [55]:
security_data.columns

Index(['SecurityID', 'OfficialName', 'SecurityType', 'SecuritySubType',
       'SecurityBBGType', 'SecurityBBGYellowKey', 'Series', 'SecuritySector',
       'SecurityIndustry', 'SecuritySubIndustry', 'BBGSecuritySector',
       'IssuerParent', 'Obligor', 'Ticker', 'EquityTicker', 'Issuer',
       'ObligorID', 'CountryDomicile', 'CountryRisk', 'CountryIssue',
       'Currency', 'Coupon', 'CouponType', 'CouponFrequency', 'CouponDayCount',
       'CreationDate', 'LastModifiedTime', 'IssueDate', 'PricingDate',
       'MaturityDate', 'IsCallable', 'IsConvertible', 'IsContingent',
       'IsFactorable', 'IsHybrid', 'IsMaple', 'IsPrivatePlacement',
       'IsPreliminary', 'IsPerpetual', 'IsJunior', 'Seniority', 'PaymentRank',
       'IsBailIn', 'QuoteSize', 'AmountIssued', 'MinIncrement', 'MinPiece',
       'BBGDescription'],
      dtype='object')

In [56]:
security_data.Seniority.unique()

array(['Sr Unsecured', 'Subordinated Unsecured',
       'Jr Subordinated Unsecured', 'Secured', '1st Lien Secured',
       '2nd Lien Secured', 'Asset Backed', '3rd Lien Secured', '1st lien',
       '1.5 Lien Secured'], dtype=object)

In [109]:
df = pd.concat([security_data['SecurityID'],pd.get_dummies(security_data.Seniority)],axis=1) 

In [110]:
df = pd.concat([df,pd.get_dummies(security_data.SecuritySector)],axis=1) 

In [111]:
df = pd.concat([df,pd.get_dummies(security_data.CountryRisk)],axis=1) 

In [112]:
df = pd.concat([df,pd.get_dummies(security_data.CouponType)],axis=1) 

In [113]:
df2 = security_data[['SecurityID','Coupon','IsCallable', 'IsConvertible', 'IsContingent',
       'IsFactorable', 'IsHybrid', 'IsMaple', 'IsPrivatePlacement',
       'IsPreliminary', 'IsPerpetual', 'IsJunior']]

In [114]:
df2.head()

,SecurityID,Coupon,IsCallable,IsConvertible,IsContingent,IsFactorable,IsHybrid,IsMaple,IsPrivatePlacement,IsPreliminary,IsPerpetual,IsJunior
0,62,4.875,0,0,0,0,0,0,0,0,0,0
1,64,5.000,1,0,0,0,0,0,1,0,0,0
2,69,1.500,0,0,0,0,0,0,0,0,0,0
3,72,6.100,0,0,0,0,0,0,0,0,0,0
4,78,5.000,0,0,0,0,0,0,0,0,0,0


## Scale MinIncrement and MinPiece

In [115]:
df3 = security_data[['MinIncrement', 'MinPiece']]

In [183]:
from sklearn.preprocessing import MinMaxScaler

In [184]:
column_names_to_normalize = ['MinIncrement','MinPiece']

In [185]:
scaler = MinMaxScaler([0,1])

In [186]:
scaler.fit(df3)

MinMaxScaler(feature_range=[0, 1])

In [187]:
df3 = pd.DataFrame(scaler.transform(df3), columns=column_names_to_normalize)

In [188]:
df_scaled = pd.concat([df,df3],axis=1) 

In [189]:
df_scaled

,SecurityID,1.5 Lien Secured,1st Lien Secured,1st lien,2nd Lien Secured,3rd Lien Secured,Asset Backed,Jr Subordinated Unsecured,Secured,Sr Unsecured,...,Fixed,PAY-IN-KIND,STEP CPN,VARIABLE,WHEN ISSUED,ZERO COUPON,MinIncrement,MinPiece,MinIncrement,MinPiece
0,62,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,50000.0,0.000005,0.000258
1,64,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,2000.0,0.000005,0.000010
2,69,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,100000.0,100000.0,0.000517,0.000517
3,72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1000.0,2000.0,0.000005,0.000010
4,78,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,150000.0,0.000005,0.000775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28971,72948,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,200000.0,0.000005,0.001033
28972,72949,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1000.0,200000.0,0.000005,0.001033
28973,72950,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1000.0,200000.0,0.000005,0.001033
28974,72951,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,200000.0,200000.0,0.001033,0.001033


In [ ]:
df_scaled.drop(columns=['\tFIXED'],inplace=True)

In [ ]:
df_unsacled.to_csv('Security_info_post_dummy_scaled.csv')

## Produce Unscaled Security Dummpy

In [147]:
df_unsacled = pd.concat([df,security_data[['MinIncrement', 'MinPiece']]],axis=1) 

In [149]:
df_unsacled.to_csv('Security_info_post_dummy.csv')

In [ ]:
final.drop(columns=['Group'],inplace= True)

In [92]:
final

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,G_change
2199,56535,2019-01-02,26.97470,1.85247,3.000000e+08,0.002132,0.014612
2200,56535,2019-01-03,26.74320,1.84797,3.000000e+08,-0.005914,0.007046
2201,56535,2019-01-04,26.95560,1.84352,3.000000e+08,0.006400,0.017257
2202,56535,2019-01-07,27.04120,1.84036,3.000000e+08,-0.001467,-0.045289
2203,56535,2019-01-08,26.90410,1.83947,3.000000e+08,-0.007478,-0.005836
...,...,...,...,...,...,...,...
528328,64,2019-02-22,5.66178,6.11500,1.697270e+09,0.019700,0.005560
528329,64,2019-02-25,5.62156,6.11537,1.697270e+09,-0.015824,-0.014690
528330,64,2019-02-26,5.62724,6.11243,1.697270e+09,0.010259,-0.002174
528331,64,2019-02-27,5.65731,6.10267,1.697270e+09,-0.006607,-0.004593


In [79]:
scaler2 = MinMaxScaler([0,1])
scaler2.fit(final.loc[:,'YieldWorst':'G_change'])

MinMaxScaler(feature_range=[0, 1])

In [125]:
df4 = scaler2.transform(final.loc[:,'YieldWorst':'G_change'])

## Spread change scaled

In [126]:
final

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,G_change
2199,56535,2019-01-02,26.97470,1.85247,3.000000e+08,0.002132,0.014612
2200,56535,2019-01-03,26.74320,1.84797,3.000000e+08,-0.005914,0.007046
2201,56535,2019-01-04,26.95560,1.84352,3.000000e+08,0.006400,0.017257
2202,56535,2019-01-07,27.04120,1.84036,3.000000e+08,-0.001467,-0.045289
2203,56535,2019-01-08,26.90410,1.83947,3.000000e+08,-0.007478,-0.005836
...,...,...,...,...,...,...,...
528328,64,2019-02-22,5.66178,6.11500,1.697270e+09,0.019700,0.005560
528329,64,2019-02-25,5.62156,6.11537,1.697270e+09,-0.015824,-0.014690
528330,64,2019-02-26,5.62724,6.11243,1.697270e+09,0.010259,-0.002174
528331,64,2019-02-27,5.65731,6.10267,1.697270e+09,-0.006607,-0.004593


In [127]:
column_names_to_normalize2 = ['YieldWorst','ModifiedDuration_Plain','AmtOutstanding','Target_G_change','G_change']

In [128]:
df4 = pd.DataFrame(df4, columns=column_names_to_normalize2,index=final.index)

In [129]:
df4

,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,G_change
2199,0.840176,0.040891,0.017689,0.643287,0.394168
2200,0.839262,0.040764,0.017689,0.643216,0.390362
2201,0.840100,0.040639,0.017689,0.643324,0.395498
2202,0.840438,0.040550,0.017689,0.643255,0.364036
2203,0.839897,0.040525,0.017689,0.643202,0.383881
...,...,...,...,...,...
528328,0.756102,0.160886,0.141270,0.643442,0.389614
528329,0.755944,0.160896,0.141270,0.643128,0.379428
528330,0.755966,0.160814,0.141270,0.643358,0.385724
528331,0.756085,0.160539,0.141270,0.643209,0.384507


In [132]:
df4 = pd.concat([final[['SecurityID','KeyDate']],df4],axis=1) 

In [133]:
df4.head()

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,G_change
2199,56535,2019-01-02,0.840176,0.040891,0.017689,0.643287,0.394168
2200,56535,2019-01-03,0.839262,0.040764,0.017689,0.643216,0.390362
2201,56535,2019-01-04,0.840100,0.040639,0.017689,0.643324,0.395498
2202,56535,2019-01-07,0.840438,0.040550,0.017689,0.643255,0.364036
2203,56535,2019-01-08,0.839897,0.040525,0.017689,0.643202,0.383881


In [166]:
##security query
Market_query = "select * from market_data_weekly"
Market_data = pd.read_sql(Market_query,con=db_connection)

In [167]:
Market_data = Market_data.fillna(0)

In [168]:
columns_name =['skew_change','swap_change','vix_change','Close_change','slope_change',
                'skew_change_Weekly','swap_change_Weekly','vix_change_Weekly','Close_change_Weekly',	
                'slope_change_Weekly']

Fill NA with 0

In [169]:
scaler3 = MinMaxScaler([0,1])
scaler3.fit(Market_data.loc[:,'skew_change':'slope_change_Weekly'])

MinMaxScaler(feature_range=[0, 1])

In [170]:
df5 = scaler3.transform(Market_data.loc[:,'skew_change':'slope_change_Weekly'])

In [171]:
df5 = pd.DataFrame(df5, columns=columns_name)

In [172]:
df5 = pd.concat([Market_data[['KeyDate']],df5],axis=1) 

In [173]:
df5

,KeyDate,skew_change,swap_change,vix_change,Close_change,slope_change,skew_change_Weekly,swap_change_Weekly,vix_change_Weekly,Close_change_Weekly,slope_change_Weekly
0,2019-01-02,0.380745,0.811007,0.206366,0.566810,0.190476,0.419945,0.773086,0.262703,0.528156,0.409717
1,2019-01-03,0.186292,0.816496,0.462358,0.445008,0.265625,0.419945,0.773086,0.262703,0.528156,0.411538
2,2019-01-04,0.368761,0.802493,0.103491,0.721569,0.250000,0.419945,0.773086,0.262703,0.528156,0.402308
3,2019-01-07,0.500301,0.822513,0.329019,0.593683,0.250000,0.419945,0.773086,0.262703,0.528156,0.411099
4,2019-01-08,0.481556,0.839151,0.266778,0.606248,0.220588,0.332999,0.773086,0.131741,0.605724,0.380149
...,...,...,...,...,...,...,...,...,...,...,...
410,2020-08-18,0.403960,0.816496,0.327708,0.571653,0.250000,0.419945,0.773086,0.262703,0.575426,0.434286
411,2020-08-19,0.403960,0.816496,0.327708,0.540260,0.250000,0.419945,0.773086,0.262703,0.523623,0.434286
412,2020-08-20,0.403960,0.816496,0.327708,0.575656,0.250000,0.419945,0.773086,0.262703,0.538279,0.434286
413,2020-08-21,0.403960,0.816496,0.327708,0.576978,0.250000,0.419945,0.773086,0.262703,0.548450,0.434286


In [174]:
final_change_data = df4.merge(df5,on=['KeyDate'],how='left')

## Unscaled Data

In [177]:
final_change_data2 = final.merge(Market_data,on=['KeyDate'],how='left')

In [176]:
final_change_data.to_csv('Security_and_market_movement_scaled_cluster24.csv')

In [178]:
final_change_data2.to_csv('Security_and_market_movement_unscaled_cluster24.csv')